In [1]:
import pandas as pd
from bokeh.models import ColumnDataSource, HoverTool, Range1d, CustomJS, Slider
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column

In [2]:
df = pd.read_csv("../data/processed/labeled-dataset.csv")
df = df.loc[df["date"].notnull()].copy()
df["year"] = pd.to_datetime(df["date"]).dt.year.astype(int)

In [3]:
colors = {label: color for label, color in zip(df["labels"].unique(), Category20.get(20))}
df["colors"] = df["labels"].map(colors)

In [4]:
source = ColumnDataSource(
    data={
        "x": df["x"],
        "y": df["y"],
        "colors": df["colors"],
        "topic": df["labels"],
        "title": df["title"],
        "author": df["author"],
        "year": df["year"],
        "alpha": [0.7] * df.shape[0],
        "size": [7] * df.shape[0]
    }
)

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Topic:</span>
            <span style="font-size: 12px">@topic<br></span>
            <span style="font-size: 12px; font-weight: bold;">Title:</span>
            <span style="font-size: 12px">@title<br></span>
            <span style="font-size: 12px; font-weight: bold;">Author:</span>
            <span style="font-size: 12px">@author<br></span>   
            <span style="font-size: 12px; font-weight: bold;">Year:</span>
            <span style="font-size: 12px">@year<br></span>         
        </div>
    </div>
    """)

In [5]:
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset']
plot = figure(
    plot_width=800, 
    plot_height=800, 
    tools=tools_emb, 
    title='UMAP projection of 2D embeddings')
plot.circle(
    "x", 
    "y", 
    size="size", 
    fill_color="colors",
    alpha="alpha", 
    line_alpha=0, 
    line_width=0.01, 
    source=source, 
    name="df", 
    legend="topic"
)

plot.legend.location = "bottom_left"
plot.legend.label_text_font_size= "8pt"
plot.legend.background_fill_alpha = 0.0
plot.legend.spacing = -5

# plot.x_range = Range1d(-10, 10)
plot.x_range = Range1d(-2, 17)
plot.y_range = Range1d(-5, 15)
# plot.y_range = Range1d(-10, 10)

In [6]:
callback = CustomJS(args=dict(source=source), code=
    """
    var data = source.data
    console.log(data)
    var f = cb_obj.value
    var x = data['x']
    var y = data['y']
    var colors = data['color']
    var alpha = data['alpha']
    var title = data['title']
    var year = data['year']
    var size = data['size']
    for (var i = 0; i < x.length; i++) {
        if (year[i] <= f) {
            alpha[i] = 0.9
            size[i] = 7
        } else {
            alpha[i] = 0.05
            size[i] = 4
        }
    }
    source.change.emit();
    """)

slider = Slider(start=df.year.min(), end=df.year.max()+1, value=2020, step=1, title="Before year")
slider.js_on_change("value", callback)

layout = column(slider, plot)
# show(layout)
output_file("../reports/figures/bokeh.html")
save(layout)

'C:\\github\\deterrence-unsupervised-top2vec\\reports\\figures\\interactive_callbacks.html'